In [1]:
import pickle
import numpy as np
from s3_train import build_model
from utils import MAX_LENGTH, LABEL_DICT

/home/qhduan/Work/cpu/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
inverse_ld = {v: k for k, v in LABEL_DICT.items()}

In [35]:
data = pickle.load(open('data.pkl', 'rb'))

In [3]:
ws = pickle.load(open('ws.pkl', 'rb'))

In [9]:
shape = (MAX_LENGTH,)

In [10]:
model, atten_model = build_model(shape, len(ws))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     3092100     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 200, 4)       1204        embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 200, 4)       2404        embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_17 

In [11]:
model.load_weights('./model_weights.hdf5')

In [44]:
def test(sentence):
    vec = ws.transform(sentence, MAX_LENGTH)
#     print(vec)
    x = np.array([vec])
    pred = model.predict(x)
    atten = atten_model.predict(x)
    atten = atten.reshape((MAX_LENGTH, 6))
    atten = np.sum(atten, axis=-1)
    print(atten.shape)
    
    ret = []
    for i in range(len(LABEL_DICT)):
        ret.append((
            inverse_ld[i],
            pred[0][i] * 100
        ))
    ret = sorted(ret, key=lambda x: x[1], reverse=True)
    for r in ret:
        print('{}\t\t{:.2f}%'.format(
            r[0],
            r[1]
        ))
#     for i, w in enumerate(list(sentence)):
#         print(atten[i], w)

In [94]:
def random_test(data):
    ind = np.random.randint(0, len(data))
    x = data[ind][0]
    y = data[ind][2]
    print(x)
    print(y)
    x = x[:MAX_LENGTH]
    test(x)

In [95]:
random_test(data)

妈妈问：皮皮你干嘛去？皮皮：和金金到家里玩啊！妈妈笑问：你带钥匙了吗？皮皮：啊，忘了！哈，臭小子，四岁就知道往家里带姑娘！不过下次记住了，往家里带妞别忘了带家门钥匙！
happy
(200,)
happy		54.72%
disgust		18.35%
neutral		9.71%
angry		6.93%
sad		6.22%
fear		2.92%
suprise		1.15%
